# Image Classification with TensorFlow
This notebook demonstrates training an image classification model using TensorFlow on CPU, saving/loading the model, and running inference.

In [1]:
# Install TensorFlow if not already installed
try:
    import tensorflow as tf
except ImportError:
    !pip install tensorflow
    import tensorflow as tf

In [2]:
# Import required libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

## Load Dataset
Images should be organized in subfolders under `datasets/images`, one folder per class.

In [5]:
import tensorflow as tf

train_dir = r"C:\Users\samri\cod\git\Farmer\Machine_Learning\Crop Quality Grading\train"
test_dir = r"C:\Users\samri\cod\git\Farmer\Machine_Learning\Crop Quality Grading\test"

img_height, img_width = 180, 180
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Classes:", class_names)


Found 15628 files belonging to 3 classes.
Found 3898 files belonging to 3 classes.
Classes: ['Bad Quality_Fruits', 'Good Quality_Fruits', 'Mixed Qualit_Fruits']


## Visualize Sample Images

## Build the Model

In [6]:
num_classes = len(class_names)

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    # tf.keras.layers.Conv2D(512, 3, activation='relu'),
    # tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

c:\Users\samri\cod\git\Farmer\Machine_Learning\venv\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Train the Model (on CPU)

In [5]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Force TensorFlow to use CPU

# Define callbacks
early_stop = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,          # Stop after 3 epochs of no improvement
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    'best_model.h5',     # File to save the best model
    monitor='val_loss',
    save_best_only=True
)

# Train the model
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[early_stop, model_checkpoint]
)


Epoch 1/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 0s 598ms/step - accuracy: 0.7337 - loss: 0.6079

489/489 ━━━━━━━━━━━━━━━━━━━━ 318s 647ms/step - accuracy: 0.8136 - loss: 0.4545 - val_accuracy: 0.9020 - val_loss: 0.2617
Epoch 2/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 0s 812ms/step - accuracy: 0.9000 - loss: 0.2780

489/489 ━━━━━━━━━━━━━━━━━━━━ 419s 857ms/step - accuracy: 0.9137 - loss: 0.2335 - val_accuracy: 0.9189 - val_loss: 0.2055
Epoch 3/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9289 - loss: 0.1925

489/489 ━━━━━━━━━━━━━━━━━━━━ 575s 1s/step - accuracy: 0.9420 - loss: 0.1559 - val_accuracy: 0.9577 - val_loss: 0.1172
Epoch 4/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 272s 556ms/step - accuracy: 0.9607 - loss: 0.1083 - val_accuracy: 0.9297 - val_loss: 0.2165
Epoch 5/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 303s 619ms/step - accuracy: 0.9719 - loss: 0.0740 - val_accuracy: 0.9418 - val_loss: 0.1868
Epoch 6/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 0s 546ms/step - accuracy: 0.9755 - loss: 0.0668

489/489 ━━━━━━━━━━━━━━━━━━━━ 290s 591ms/step - accuracy: 0.9804 - loss: 0.0539 - val_accuracy: 0.9695 - val_loss: 0.1103
Epoch 7/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 308s 564ms/step - accuracy: 0.9866 - loss: 0.0394 - val_accuracy: 0.9659 - val_loss: 0.1304
Epoch 8/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 1857s 4s/step - accuracy: 0.9915 - loss: 0.0237 - val_accuracy: 0.9710 - val_loss: 0.1263
Epoch 9/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 481s 984ms/step - accuracy: 0.9880 - loss: 0.0354 - val_accuracy: 0.9728 - val_loss: 0.1185


## Load the Model

In [7]:
loaded_model = tf.keras.models.load_model('best_model.h5')

## Inference on a New Image

In [8]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

img_height, img_width = 180, 180

def predict_image(img_path):
    img = load_img(img_path, target_size=(img_height, img_width))
    img_array = img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    predictions = loaded_model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    predicted_class = class_names[np.argmax(score)]
    confidence = 100 * np.max(score)
    print(f"Predicted class for the fruits are: {predicted_class} ({confidence:.2f}% confidence)")
    return predicted_class, confidence

test_image_paths = [
    r"C:\Users\samri\cod\git\Farmer\Machine_Learning\Crop Quality Grading\test\Bad Quality_Fruits\Lime_Bad\Lime_Bad_176.jpg",
    r"C:\Users\samri\cod\git\Farmer\Machine_Learning\Crop Quality Grading\test\Good Quality_Fruits\Orange_Good\Orange_Good_27.jpg",
    r"C:\Users\samri\cod\git\Farmer\Machine_Learning\Crop Quality Grading\test\Mixed Qualit_Fruits\Banana\Banana_2.jpg"
]

for img_path in test_image_paths:
    predict_image(img_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
Predicted class for the fruits are: Bad Quality_Fruits (83.51% confidence)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted class for the fruits are: Good Quality_Fruits (88.32% confidence)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Predicted class for the fruits are: Mixed Qualit_Fruits (99.89% confidence)
